In [1]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [2]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
figsize=(10,4)


In [3]:
def rename_model_names(df):
    """Rename model names in the DataFrame for better readability."""
    df = df.replace({
        "Model": {
            "B=1_pretrained_CE_bcosified": "B=2 (Bcosified Model)",
            "B=1_pretrained_CE-parallel-adapters-B=2_B=2": "B=2 Model with B=2 Adapter",
            "B=1_pretrained_CE-parallel-residual-adapters-B=2_B=2": "B=2 Model with B=2 Adapter"
        }
    })
    return df

In [ ]:
conv_adapter = df[df['group'] == "vit-lora"]
conv_adapter = conv_adapter[conv_adapter["config_conv_adapter_act_layer"] == "relu"]
conv_adapter = conv_adapter[conv_adapter["config_lora_use_neat"] != True]
conv_adapter = conv_adapter[conv_adapter["config_batch_size"] == 128]
conv_adapter = conv_adapter[conv_adapter["state"] != "crashed"]  # Filter for finished runs
conv_adapter = conv_adapter[conv_adapter["config_b"] == 2]  # Filter for finished runs
conv_adapter = conv_adapter[conv_adapter["name"] == "vit_b_16_B=2_pretrained_lora_relu"]  # Filter for finished runs
conv_adapter = conv_adapter[conv_adapter["config_lora_initialization_mode"] != "kaiming-uniform"]  # Filter for finished runs
vit_base = df[df['id'] == "2fupnc3h"] # Base model
conv_adapter = pd.concat([conv_adapter, vit_base], ignore_index=True)  # Add the Bcosified model to the DataFrame
# conv_wo_batchnorm = conv_wo_batchnorm[conv_wo_batchnorm["id"].isin(["sbpfkwct", "dv8y7o8k", "yheexwdv", "zf9oo2l6", "xka2n8rp"])]  # Filter for specific runs
# conv_wo_batchnorm = rename(conv_wo_batchnorm)  # Rename model names for better readability
conv_adapter.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(conv_adapter)}")

#Renaming
conv_adapter = pu.rename(conv_adapter).sort_values(by=val_accuracy, ascending=False)
conv_adapter = rename_model_names(conv_adapter)  # Rename model names for better readability
# conv_adapter = pu.add_trainable_layers_column(conv_adapter, frozen_layers_col=frozen_layers, new_col_name=trainable_layers)
conv_adapter = pu.rename_adapter_factor_experiments(conv_adapter)  # Rename columns for adapter factor experiments

# Some stats
for name in conv_adapter["Model"].unique(): print(name)  # Print each model name
display(conv_adapter)


Number of runs in layerwise_freezing group: 4
vit_b_16_B=2_pretrained
vit_b_16_B=2_pretrained_lora_relu


id                              Model     state        entity  \
3  2fupnc3h            vit_b_16_B=2_pretrained  finished  raphaelmaser   
2  il3wbf8o  vit_b_16_B=2_pretrained_lora_relu    failed  raphaelmaser   
0  f96i2dw8  vit_b_16_B=2_pretrained_lora_relu  finished  raphaelmaser   
1  w71nmjrt  vit_b_16_B=2_pretrained_lora_relu  finished  raphaelmaser   

                                  project            created_at  updated_at  \
3  Improved-Interpretability-and-Concepts  2025-07-19T06:48:52Z         NaN   
2  Improved-Interpretability-and-Concepts  2025-07-17T11:11:50Z         NaN   
0  Improved-Interpretability-and-Concepts  2025-07-16T16:14:29Z         NaN   
1  Improved-Interpretability-and-Concepts  2025-07-16T10:51:14Z         NaN   

               tags notes                                                url  \
3               NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
2               NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0               NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1  LAYERNORM_FROZEN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

      group  job_type  sweep                   synced_at  config_b  config_lr  \
3  vit-base       NaN    NaN  2025-07-24T10:48:37.390495       2.0     0.0010   
2  vit-lora       NaN    NaN  2025-07-24T10:48:02.990999       2.0     0.0001   
0  vit-lora       NaN    NaN  2025-07-24T10:47:50.617145       2.0     0.0010   
1  vit-lora       NaN    NaN  2025-07-24T10:46:48.847028       2.0     0.0010   

  config_loss  config_seed config_tags config_group config_model  \
3  UniformBCE         42.0         NaN     vit-base          vit   
2  UniformBCE         42.0         NaN     vit-lora          vit   
0  UniformBCE         42.0         NaN     vit-lora          vit   
1  UniformBCE         42.0         NaN     vit-lora          vit   

   config_epochs config_adapter  config_compile config_dataset  \
3            100            NaN            True   TinyImagenet   
2            100            NaN            True   TinyImagenet   
0            100            NaN            True   TinyImagenet   
1            100            NaN            True   TinyImagenet   

   config_vit_dim config_use_lora config_act_layer  config_b_adapter  \
3           768.0           False             relu               NaN   
2           768.0            True             relu               NaN   
0           768.0            True             relu               NaN   
1           768.0            True             relu               NaN   

                                    config_data_path config_precision  \
3  /home/rmaser/rmaser_improved_interpretability_...       bf16-mixed   
2  /home/rmaser/rmaser_improved_interpretability_...       bf16-mixed   
0  /home/rmaser/rmaser_improved_interpretability_...       bf16-mixed   
1  /home/rmaser/rmaser_improved_interpretability_...       bf16-mixed   

   config_vit_depth  config_vit_qkv_b  config_batch_size config_cache_data  \
3              12.0               1.0                128              True   
2              12.0               1.0                128              True   
0              12.0               1.0                128              True   
1              12.0               1.0                128              True   

                                   config_experiment  config_image_size  \
3  <experiment.experiment.Experiment object at 0x...              224.0   
2  <experiment.experiment.Experiment object at 0x...              224.0   
0  <experiment.experiment.Experiment object at 0x...              224.0   
1  <experiment.experiment.Experiment object at 0x...              224.0   

  config_norm_layer config_pin_memory config_block_class config_convolution  \
3         layernorm              True    bottleneck_relu         BcosConv2d   
2         layernorm              True    bottleneck_relu         BcosConv2d   
0         layernorm             

In [7]:
generate_scatter_plot_accuracy(
    data=conv_adapter,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViT LoRA",
    connect_dots=False,
    scale_dots_by_trainable_params=trainable_params,
    range_y=(0,0.9)
)

In [10]:
create_ultimate_plotly_plot(
    data=conv_adapter,
    x=trainable_params,
    y=val_accuracy,
    plot_type="bar",
    title="ViT LoRA",
    range_y=(0,0.9)
)